In [1]:
from keys import NEWS_API_KEY, OPENAI_API_KEY, GOOGLE_API_KEY, GOOGLE_CSE_ID, WOLFRAM_ALPHA_APPID, WEATHER_API_KEY
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents import load_tools
from langchain.agents import Tool
import time

# Video Recording
[Link to video here](https://youtu.be/7VOPEAVuTtM)

# Experimenting with the Agent Tools
Let's start by seeing how we can influence the behavior of an Agent by changing it's available tooling. Agents are ultimately decision makers about when to use Tools and which Tools to use.

For this example, we'll only give the Agent access to Wikipedia by injecting Wikipedia into the Agent prompt.

In [2]:
TEMPLATE = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When using the CSE Lookup tool, respond with the first result you receive from the tool. Do not use Google Search or Wikipedia to answer questions about the CSE course catalog.

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```

SUFFIX = Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}"""


In [3]:

class Chatbot:
    def __init__(self, template=TEMPLATE):
        self.template = template
        self.prompt = PromptTemplate(
            input_variables=["tool_names", "ai_prefix",
                             "chat_history", "agent_scratchpad", "input"],
            template=self.template)
        self.memory = ConversationBufferMemory()
        self.llm = ChatOpenAI(
            openai_api_key=OPENAI_API_KEY,
            temperature=0,
            model_name='gpt-3.5-turbo')
        self.tools = load_tools([
            'wikipedia', ################# only wikipedia
        ],
            llm=self.llm)
        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            memory=self.memory,
            prompt=self.prompt)
    def respond(self, user_prompt):
        return self.agent.run(input=user_prompt)
    
chatbot = Chatbot(template=TEMPLATE)
chatbot.respond("What is the weather like today?")




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Wikipedia",
  "action_input": {
    "query": "weather today"
  }
}
```

Observation: Page: Weather
Summary: Weather is the state of the atmosphere, describing for example the degree to which it is hot or cold, wet or dry, calm or stormy, clear or cloudy. On Earth, most weather phenomena occur in the lowest layer of the planet's atmosphere, the troposphere, just below the stratosphere. Weather refers to day-to-day temperature, precipitation, and other atmospheric conditions, whereas climate is the term for the averaging of atmospheric conditions over longer periods of time. When used without qualification, "weather" is generally understood to mean the weather of Earth.
Weather is driven by air pressure, temperature, and moisture differences between one place and another. These differences can occur due to the Sun's angle at any particular spot, which varies with latitude. The strong temperature contrast between polar and

'I recommend using a weather website or app to get the most accurate and up-to-date information on the current weather in your location.'

When you're a hammer, everything looks like a nail. Wikipedia is great, but it doesn't give us the answer we're looking for. Let's add a weather Tool to the Agent.

The Agent will see the Tool listed in its initial prompt and will search over the natural language descriptions of those tools to determine which to use. Here is what the Agent sees for the OpenWeatherMap API Tool:

``` 
    name = "OpenWeatherMap"
    description = (
        "A wrapper around OpenWeatherMap API. "
        "Useful for fetching current weather information for a specified location. "
        "Input should be a location string (e.g. London,GB)."
    )
````

In [4]:
class Chatbot:
    def __init__(self, template=TEMPLATE):
        self.template = template
        self.prompt = PromptTemplate(
            input_variables=["tool_names", "ai_prefix",
                             "chat_history", "agent_scratchpad", "input"],
            template=self.template)
        self.memory = ConversationBufferMemory()
        self.llm = ChatOpenAI(
            openai_api_key=OPENAI_API_KEY,
            temperature=0,
            model_name='gpt-3.5-turbo')
        self.tools = load_tools([
            'wikipedia',
            'openweathermap-api'
        ],
            llm=self.llm,
            openweathermap_api_key=WEATHER_API_KEY)
        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            memory=self.memory,
            prompt=self.prompt)
    def respond(self, user_prompt):
        return self.agent.run(input=user_prompt)
    
chatbot = Chatbot(template=TEMPLATE)
chatbot.respond("What is the weather like today?")



> Entering new AgentExecutor chain...
Thought: I can use the OpenWeatherMap tool to fetch the current weather information for the user's location.

Action:
```
{
  "action": "OpenWeatherMap",
  "action_input": "New York"
}
```


Observation: In New York, the current weather is as follows:
Detailed status: clear sky
Wind speed: 7.15 m/s, direction: 331°
Humidity: 35%
Temperature: 
  - Current: 24.79°C
  - High: 27.9°C
  - Low: 19.41°C
  - Feels like: 24.24°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought:The current weather in New York is clear and sunny with a temperature of 24.79°C. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "The current weather in New York is clear and sunny with a temperature of 24.79°C."
}
```



> Finished chain.


'The current weather in New York is clear and sunny with a temperature of 24.79°C.'


# Experimenting with Vector Database Retrieval


There are a few parameters can can be tweaked when it comes to retrieval augmentation. In this demo, we'll explore _k_ chunks being returned to the `RetrievalQA` chain. 

If _k_ is low, only a few docs/chunks will be returned and the LLM will need to use those to determine an answer. The more chunks _k_ that we return, the better the reponse, but the lower the performance. This is especially tricky for the UW CSE course catalog where docs (classes) much later down the page and therefore in separate chunks may reference information much earlier in the page (e.g., course prerequisites).

We'll explore an example of a low _k_, followed by a high _k_, looking at both their reponse and the time it took to generate it.

In [9]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Chroma(persist_directory='rtdocs-vectordb',
                  embedding_function=embeddings)

# IMPORTANT
# search_type "mmr" or "similarity", where mmr aims for both relevance and diversity
# k is the number of results to return, the more results we return the better the extraction between vectors,
# but we take a hit to performance especially when using map_reduce chain_type
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 6}) 

# IMPORTANT
# map_reduce passes the inital prompt to each vector chunk that the retriever retrieves. 
# it then aggregates the responses from each chunk and passes those back to itself to generate a final response
# there are other chain_types, but map_reduce is the most accurate for our use case, 
# please see final report for more details
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0),
                                       chain_type="map_reduce",
                                       retriever=retriever,
                                       return_source_documents=False) #if you set to true, 
#you will need to change the Tool to use qa.predict instead and parse the args 

Using embedded DuckDB with persistence: data will be stored in: rtdocs-vectordb


In [10]:
TEMPLATE = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------
- UW CSE Lookup


To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When using the CSE Lookup tool, respond with the first result you receive from the tool. Do not use Google Search or Wikipedia to answer questions about the CSE course catalog.

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```

SUFFIX = Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}"""


class Chatbot:
    def __init__(self, template=TEMPLATE, qa_chain=qa_chain):
        self.template = template
        self.prompt = PromptTemplate(
            input_variables=["tool_names", "ai_prefix",
                             "chat_history", "agent_scratchpad", "input"],
            template=self.template)
        self.memory = ConversationBufferMemory()
        self.llm = ChatOpenAI(
            openai_api_key=OPENAI_API_KEY,
            temperature=0,
            model_name='gpt-3.5-turbo')
        # load_tools() is not compatible with custom tools
        self.tools = [Tool(
                name="UW CSE Lookup",
                func=qa_chain.run,
                description=("use this tool when you need to answer questions "
                             "about the UW CSE course catalog "
                             "or CSE courses in general "
                             "input should be a fully formed question"),
                return_direct=True # add return_direct=True if you want to return the Tool response rather than the agent response
            )]
        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            memory=self.memory,
            prompt=self.prompt)

    def respond(self, user_prompt):
        return self.agent.run(input=user_prompt)
    


In [11]:
chatbot = Chatbot(template=TEMPLATE, qa_chain=qa_chain)
start = time.time()
chatbot.respond("What is CSE 311 about?")
stop = time.time()
print(stop - start)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What is the course description for CSE 311?"
}
```

Observation: I'm sorry, I cannot provide an answer to this question as the course description for CSE 311 was not provided in the given text.


> Finished chain.
22.10927939414978


In [19]:
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 12}) 

qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0),
                                       chain_type="map_reduce",
                                       retriever=retriever,
                                       return_source_documents=False)

chatbot = Chatbot(template=TEMPLATE, qa_chain=qa_chain)
start = time.time()
chatbot.respond("What is CSE 311 about?")
stop = time.time()
print(stop - start)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What is the course content of CSE 311?"
}
```

Observation: Here's the relevant text:

CSE 311: Foundations of Computing I    
          Basic concepts of computer organization, including machine and assembly language, and operating systems. Number systems, Boolean algebra, and digital logic. Data structures, algorithms, and program design using C. Prerequisite: CSE 143.

The course content of CSE 311 includes basic concepts of computer organization, including machine and assembly language, and operating systems. It also covers number systems, Boolean algebra, and digital logic. Additionally, the course covers data structures, algorithms, and program design using C.


> Finished chain.
43.107470989227295


**_NOTE TO THE READER:_**  

This is wildly inconsistent behavior! The vectorDB interaction with the LLM does **NOT** work like a typical database. Each run of both MapReduce algorithm produces outputs, which are in turn consumed as inputs by the LLM. There can be a huge deviation in the content and the quality of content between each iterative step of these algorithms. MMR is expected to be very consistent though, since it is a mathematical calculation on the vector embedding.

You may find that the LLM fails to produce a higher quality answer (just try re-running the cell above) but it will still take much longer.


# Experimenting with Custom Tools


Let's see what happens when we change the natural language description of a CustomTool. The Agent will be using the description as input to it's behavior. Be sure to look at the `self.tools = Tool(description=)` lines.

In [20]:
TEMPLATE = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------
- Wikipedia
- Math
- Google Search
- Python REPL
- Wolfram Alpha
- Terminal
- News API
- OpenWeatherMap API
- UW CSE Lookup


To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When using the CSE Lookup tool, respond with the first result you receive from the tool. Do not use Google Search or Wikipedia to answer questions about the CSE course catalog.

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```

SUFFIX = Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}"""

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Chroma(persist_directory='rtdocs-vectordb',
                  embedding_function=embeddings)

# IMPORTANT
# search_type "mmr" or "similarity", where mmr aims for both relevance and diversity
# k is the number of results to return, the more results we return the better the extraction between vectors, but we take a hit to performance especially when using map_reduce chain_type
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 12}) 

# IMPORTANT
# map_reduce passes the inital prompt to each vector chunk that the retriever retrieves. it then aggregates the responses from each chunk and passes those back to itself to generate a final response
# there are other chain_types, but map_reduce is the most accurate for our use case, please see final report for more details
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0),
                                       chain_type="map_reduce",
                                       retriever=retriever,
                                       return_source_documents=False) #if you set to true, you will need to change the Tool to use qa.predict instead and parse the args 


Using embedded DuckDB with persistence: data will be stored in: rtdocs-vectordb


In [25]:
class Chatbot:
    def __init__(self, template=TEMPLATE, qa_chain=qa_chain):
        self.template = template
        self.prompt = PromptTemplate(
            input_variables=["tool_names", "ai_prefix",
                             "chat_history", "agent_scratchpad", "input"],
            template=self.template)
        self.memory = ConversationBufferMemory()
        self.llm = ChatOpenAI(
            openai_api_key=OPENAI_API_KEY,
            temperature=0,
            model_name='gpt-3.5-turbo')
        self.tools = load_tools([
            'wikipedia',
            'llm-math',
            'python_repl',
            'wolfram-alpha',
            'terminal',
            'news-api',
            'openweathermap-api'
        ],
            llm=self.llm,
            news_api_key=NEWS_API_KEY,
            openweathermap_api_key=WEATHER_API_KEY,
            wolfram_alpha_appid=WOLFRAM_ALPHA_APPID,
            google_cse_id=GOOGLE_CSE_ID,
            google_api_key=GOOGLE_API_KEY)
        # load_tools() is not compatible with custom tools
        self.tools = self.tools + [Tool(
                name="UW CSE Lookup",
                func=qa_chain.run,
                description=("use this tool when you need to answer questions "
                             "about the UW CSE course catalog "
                             "or CSE courses in general "),
                return_direct=True # add return_direct=True if you want to return the Tool response rather than the agent response
            )]
        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            memory=self.memory,
            prompt=self.prompt)

    def respond(self, user_prompt):
        return self.agent.run(input=user_prompt)

chatbot = Chatbot()
chatbot.respond("Do you know any course in CSE that offers jointly with E E ?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "CSE courses jointly offered with EE"
}
``` 

Observation: There is no relevant text in the given portion of the document that mentions CSE courses jointly offered with EE.


> Finished chain.


'There is no relevant text in the given portion of the document that mentions CSE courses jointly offered with EE.'

In [26]:
class Chatbot:
    def __init__(self, template=TEMPLATE, qa_chain=qa_chain):
        self.template = template
        self.prompt = PromptTemplate(
            input_variables=["tool_names", "ai_prefix",
                             "chat_history", "agent_scratchpad", "input"],
            template=self.template)
        self.memory = ConversationBufferMemory()
        self.llm = ChatOpenAI(
            openai_api_key=OPENAI_API_KEY,
            temperature=0,
            model_name='gpt-3.5-turbo')
        self.tools = load_tools([
            'wikipedia',
            'llm-math',
            'python_repl',
            'wolfram-alpha',
            'terminal',
            'news-api',
            'openweathermap-api'
        ],
            llm=self.llm,
            news_api_key=NEWS_API_KEY,
            openweathermap_api_key=WEATHER_API_KEY,
            wolfram_alpha_appid=WOLFRAM_ALPHA_APPID,
            google_cse_id=GOOGLE_CSE_ID,
            google_api_key=GOOGLE_API_KEY)
        # load_tools() is not compatible with custom tools
        self.tools = self.tools + [Tool(
                name="UW CSE Lookup",
                func=qa_chain.run,
                description=("use this tool when you need to answer questions "
                             "about the UW CSE course catalog "
                             "or CSE courses in general "
                             "input should be a fully formed question"), ## !!! see how this line impacts
                              # "action_input" by the agent
                return_direct=True # add return_direct=True if you want to return the Tool response rather
                 # than the agent response
            )]
        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            memory=self.memory,
            prompt=self.prompt)

    def respond(self, user_prompt):
        return self.agent.run(input=user_prompt)

chatbot = Chatbot()
chatbot.respond("Do you know any course in CSE that offers jointly with E E ?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What CSE courses are jointly offered with EE?"
}
``` 

Observation: According to the given information, only CSE 561: Computer Communications And Networks is jointly offered with EE.


> Finished chain.


'According to the given information, only CSE 561: Computer Communications And Networks is jointly offered with EE.'

Obviously this answer is incorrect, but notice how the Agent passed a fully formed question to the Tool rather than a statement. It may seem inconsequential, but when natural language governs all the application logic then phrasing can really matter.

# Tool Demonstrations
Below we will highlight some of the available Tools from the langchain library. 

## Wikipedia


In [27]:
chatbot.respond("What can you tell me about the moon?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Moon"
}
```


/home/master/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/master/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Moon
Summary: The Moon is Earth's only natural satellite. Its diameter is about one-quarter of Earth's (comparable to the width of Australia), making it the fifth largest satellite in the Solar System and the largest and most massive relative to its parent planet. It is larger than all known dwarf planets in the Solar System. The Moon is a planetary-mass object with a differentiated rocky body, making it a satellite planet under the geophysical definitions of the term. It lacks any significant atmosphere, hydrosphere, or magnetic field. Its surface gravity is about one-sixth of Earth's at 0.1654 g—Jupiter's moon Io is the only satellite in the Solar System known to have a higher surface gravity and density.
The Moon orbits Earth at an average distance of 384,400 km (238,900 mi), or about 30 times Earth's diameter. Its gravitational influence is the main driver of Earth's tides and very slowly lengthens Earth's day. The Moon's orbit around Earth has a sidereal period

'The previous action provided a good summary of the Moon. Is there anything else you would like to know about it?'

## Math


In [31]:
chatbot.respond("What is 13 raised to the .3432 power?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "13 ** 0.3432"
}
```

Observation: Answer: 2.4116004626599237
Thought:The previous answer was correct. I can return it as the final answer.
Action:
```
{
  "action": "Final Answer",
  "action_input": "2.4116004626599237"
}
```


> Finished chain.


'2.4116004626599237'

## Python REPL


In [44]:
chatbot.respond("What is the output of print([x for x in range(10)])")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Python REPL",
  "action_input": "print([x for x in range(10)])"
}
```

Observation: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Thought:The human might have follow-up questions about the output. I should ask if they need any further information.
Action:
```
{
  "action": "Final Answer",
  "action_input": "[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]. Do you need any further information?"
}
```

> Finished chain.


'[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]. Do you need any further information?'

## Wolfram Alpha


In [32]:
chatbot.respond("What is 2x+5 = -3x + 7?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Wolfram Alpha",
  "action_input": "2x+5=-3x+7"
}
``` 

Observation: Assumption: 2 x + 5 = -3 x + 7 
Answer: x = 2/5
Thought:The previous answer is correct. I can return it as the final answer.
Action:
```
{
  "action": "Final Answer",
  "action_input": "x = 2/5"
}
``` 



> Finished chain.


'x = 2/5'

## Terminal
This one is experiencing a validation error that I was not able to workout, but you can see that it is legitimately interacting with my filesystem. Compare this `ls` to what you see in the repository!

In [52]:
chatbot.respond({"commands": ["pwd", "ls"], "query": "what is the output of the following commands?"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "terminal",
  "action_input": {
    "commands": ["pwd", "ls"]
  }
}
```

Observation: /home/master/code/gballew
api.py
chatbot
debugserver.py
FINAL_DEMO.ipynb
FINAL_REPORT.ipynb
keys.py
__pycache__
README.md
requirements.txt
rtdocs
rtdocs-create.ipynb
rtdocs-query.ipynb
rtdocs-vectordb

Thought:

/home/master/anaconda3/lib/python3.9/site-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


You asked for the output of the `pwd` and `ls` commands. The `pwd` command prints the current working directory, which is `/home/master/code/gballew`. The `ls` command lists the files and directories in the current directory, which are `api.py`, `chatbot`, `debugserver.py`, `FINAL_DEMO.ipynb`, `FINAL_REPORT.ipynb`, `keys.py`, `__pycache__`, `README.md`, `requirements.txt`, `rtdocs`, `rtdocs-create.ipynb`, `rtdocs-query.ipynb`, and `rtdocs-vectordb`.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The output of the `pwd` command is `/home/master/code/gballew`. The output of the `ls` command is `api.py`, `chatbot`, `debugserver.py`, `FINAL_DEMO.ipynb`, `FINAL_REPORT.ipynb`, `keys.py`, `__pycache__`, `README.md`, `requirements.txt`, `rtdocs`, `rtdocs-create.ipynb`, `rtdocs-query.ipynb`, and `rtdocs-vectordb`."
}
```



> Finished chain.


ValidationError: 1 validation error for HumanMessage
content
  str type expected (type=type_error.str)

## News API

Check out how the Agent refines its query interaction with the Tool!

In [53]:
chatbot.respond("What are the top headlines today?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "News API",
  "action_input": "What are the top headlines today?"
}
``` 

Observation: The API call was unsuccessful because required parameters are missing. The necessary parameters are sources, q, language, country, and category.
Thought:Let me modify the previous action to include the necessary parameters.
Action:
```
{
  "action": "News API",
  "action_input": "What are the top headlines today from USA?"
}
``` 


Observation: The top headlines today from the USA are about Apple unveiling their new augmented-reality headset called the Apple Vision Pro at their annual software developer conference. The headset will retail for $3,499 and will be available early next year on Apple's website. The response also includes a link to a live blog for updates on WWDC 2023.
Thought:The response seems accurate and helpful. I can return it as the final answer.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The top hea

"The top headlines today from the USA are about Apple unveiling their new augmented-reality headset called the Apple Vision Pro at their annual software developer conference. The headset will retail for $3,499 and will be available early next year on Apple's website. You can follow a live blog for updates on WWDC 2023 at https://www.cnn.com/business/live-news/apple-wwdc-2023-june-5/index.html"

## OpenWeatherMap API


In [57]:
chatbot.respond("What's the weather like in London today?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "OpenWeatherMap",
  "action_input": {
    "location": "London,GB"
  }
}
``` 

Observation: In London,GB, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 3.6 m/s, direction: 0°
Humidity: 74%
Temperature: 
  - Current: 12.8°C
  - High: 13.47°C
  - Low: 11.49°C
  - Feels like: 12.07°C
Rain: {}
Heat index: None
Cloud cover: 75%
Thought:The weather in London is currently broken clouds with a temperature of 12.8°C and a humidity of 74%. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "The weather in London is currently broken clouds with a temperature of 12.8°C and a humidity of 74%."
}
```


> Finished chain.


'The weather in London is currently broken clouds with a temperature of 12.8°C and a humidity of 74%.'

## UW CSE Lookup
This one is a custom tool, but let's use it one more time for good measure! 

In [80]:
chatbot.respond("What classes can I take after CSE 142?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What are the classes that can be taken after CSE 142?"
}
``` 

Observation: According to the given portions of the document, the classes that can be taken after CSE 142 are CSE 143: Computer Programming II, CSE 154: Web Programming, CSE 163: Intermediate Data Programming, CSE 180: Introduction to Data Science, CSE 190a: Women in CSE Seminar, CSE 190b: Direct Admission Seminar, CSE 311: Foundations Of Computing I, CSE 331: Software Design And Implementation, CSE 332: Data Structures and Parallelism, CSE 333: Systems Programming, CSE 340: Interaction Programming, and CSE 341: Programming Languages.


> Finished chain.


'According to the given portions of the document, the classes that can be taken after CSE 142 are CSE 143: Computer Programming II, CSE 154: Web Programming, CSE 163: Intermediate Data Programming, CSE 180: Introduction to Data Science, CSE 190a: Women in CSE Seminar, CSE 190b: Direct Admission Seminar, CSE 311: Foundations Of Computing I, CSE 331: Software Design And Implementation, CSE 332: Data Structures and Parallelism, CSE 333: Systems Programming, CSE 340: Interaction Programming, and CSE 341: Programming Languages.'

In [81]:
chatbot.respond("What classes have CSE 142 as a prerequisite?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What classes have CSE 142 as a prerequisite?"
}
``` 

Observation: According to the given portion of the document, the following classes have CSE 142 as a prerequisite: CSE 143: Computer Programming II, CSE 163: Intermediate Data Programming, CSE 190a: Women in CSE Seminar, CSE 131: Science and Art of Digital Photography, CSE 154: Web Development and Data, and CSE 390: Special Topics in Computer Science and Engineering.


> Finished chain.


'According to the given portion of the document, the following classes have CSE 142 as a prerequisite: CSE 143: Computer Programming II, CSE 163: Intermediate Data Programming, CSE 190a: Women in CSE Seminar, CSE 131: Science and Art of Digital Photography, CSE 154: Web Development and Data, and CSE 390: Special Topics in Computer Science and Engineering.'

In [82]:
chatbot.respond("What is CSE 142 about?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What is the course content of CSE 142?"
}
```


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID c9dc48227e613091558559696a8a338e in your message.).



Observation: The course content of CSE 142 includes basic programming-in-the-small abilities and concepts including procedural programming (methods, parameters, return, values), basic control structures (sequence, if/else, for loop, while loop), file processing, arrays, and an introduction to defining objects. It is intended for students without prior programming experience.


> Finished chain.


'The course content of CSE 142 includes basic programming-in-the-small abilities and concepts including procedural programming (methods, parameters, return, values), basic control structures (sequence, if/else, for loop, while loop), file processing, arrays, and an introduction to defining objects. It is intended for students without prior programming experience.'

In [61]:
chatbot.respond("What is the course description for CSEP 546?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What is the course description for CSEP 546?"
}
``` 

Observation: The course description for CSEP 546 is: "Methods for designing systems that learn from data and improve with experience. Supervised learning and predictive modeling; decision trees, rule induction, nearest neighbors, Bayesian methods, neural networks, support vector machines, and model ensembles. Unsupervised learning and clustering. Prerequisite: CSE PMP majors only."


> Finished chain.


'The course description for CSEP 546 is: "Methods for designing systems that learn from data and improve with experience. Supervised learning and predictive modeling; decision trees, rule induction, nearest neighbors, Bayesian methods, neural networks, support vector machines, and model ensembles. Unsupervised learning and clustering. Prerequisite: CSE PMP majors only."'

In [62]:
chatbot.respond("What is the course number for natural language processing?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What is the course number for natural language processing?"
}
``` 

Observation: The course number for natural language processing is either CSE 447, CSE 517, LING 472, or CSEP 517, depending on which part of the document is correct. There is conflicting information provided.


> Finished chain.


'The course number for natural language processing is either CSE 447, CSE 517, LING 472, or CSEP 517, depending on which part of the document is correct. There is conflicting information provided.'

In [63]:
chatbot.respond("What 300 level Software Design class can I take?")



> Entering new AgentExecutor chain...
Thought: I can use the UW CSE Lookup tool to answer this question.

Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What 300 level Software Design classes are available at UW CSE?"
}
```



Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 67c89cd14621ec6498f7a6da92840a1b in your message.).



Observation: According to the given portions of the document, there are three 300 level Software Design classes available at UW CSE: CSE 331: Software Design And Implementation, CSE 332: Data Structures and Parallelism, and CSE 333: Systems Programming.


> Finished chain.


'According to the given portions of the document, there are three 300 level Software Design classes available at UW CSE: CSE 331: Software Design And Implementation, CSE 332: Data Structures and Parallelism, and CSE 333: Systems Programming.'

In [64]:
chatbot.respond("Are there any courses about software design?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What UW CSE courses cover software design?"
}
``` 

Observation: The following UW CSE courses cover software design: CSE 481: Capstone Software Design, CSE 332, CSE 351, CSE 331, and CSE 352. Additionally, CSEP 503: Principles Of Software Engineering covers software design. CSE 331: Software Design And Implementation and CSE 504: Advanced Topics In Software Engineering also cover software design to some extent. It is possible that other courses may cover software design as well, but further information would be needed to confirm this.


> Finished chain.


'The following UW CSE courses cover software design: CSE 481: Capstone Software Design, CSE 332, CSE 351, CSE 331, and CSE 352. Additionally, CSEP 503: Principles Of Software Engineering covers software design. CSE 331: Software Design And Implementation and CSE 504: Advanced Topics In Software Engineering also cover software design to some extent. It is possible that other courses may cover software design as well, but further information would be needed to confirm this.'

In [65]:
chatbot.respond("What class should I take if I want to take CSE 546?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What class do I need to take to enroll in CSE 546?"
}
``` 

Observation: There is no information in the given portion of the document about the class required to enroll in CSE 546.


> Finished chain.


'There is no information in the given portion of the document about the class required to enroll in CSE 546.'

In [66]:
chatbot.respond("What are the prerequisites for CSE 546?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What are the prerequisites for CSE 546?"
}
``` 

Observation: Based on the given portion of the document, the prerequisites for CSE 546 are either a minimum grade of 2.5 in MATH 098, a minimum grade of 3.0 in MATH 103, a score of 151-169 on the MPT-GS placement test, or score of 145-153 on the MPT-AS placement test.


> Finished chain.


'Based on the given portion of the document, the prerequisites for CSE 546 are either a minimum grade of 2.5 in MATH 098, a minimum grade of 3.0 in MATH 103, a score of 151-169 on the MPT-GS placement test, or score of 145-153 on the MPT-AS placement test.'

Note here how the phrasing of the question directly impacts the result for the previous 2 queries. 

In [67]:
chatbot.respond("What class can I take after taking CSE 143 and CSE 351?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What classes can I take after CSE 143 and CSE 351?"
}
``` 

Observation: After completing CSE 143 and CSE 351, you can take the following classes: 
- CSE 344: Introduction To Data Management
- CSE 352: Hardware Design And Implementation
- CSE 369: Introduction to Digital Design
- CSE 373: Data Structures And Algorithms
- CSE 374: Intermediate Programming Concepts And Tools
- CSE 390a: System and Software Tools


> Finished chain.


'After completing CSE 143 and CSE 351, you can take the following classes: \n- CSE 344: Introduction To Data Management\n- CSE 352: Hardware Design And Implementation\n- CSE 369: Introduction to Digital Design\n- CSE 373: Data Structures And Algorithms\n- CSE 374: Intermediate Programming Concepts And Tools\n- CSE 390a: System and Software Tools'

In [68]:
chatbot.respond("What's the course number for the professional masters version of Computer Vision?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What is the course number for the professional masters version of Computer Vision?"
}
``` 

Observation: The course number for the professional masters version of Computer Vision is CSEP 590.


> Finished chain.


'The course number for the professional masters version of Computer Vision is CSEP 590.'

In [69]:
chatbot.respond("What's the course id of professional master version of Computer Vision?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What is the course ID for the professional master version of Computer Vision?"
}
``` 


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e22b1368c37d8934e543101bf19f2f5c in your message.).



Observation: The course ID for the professional master version of Computer Vision is CSEP 576.


> Finished chain.


'The course ID for the professional master version of Computer Vision is CSEP 576.'

In [70]:
chatbot.respond("Do you know any course in CSE that offers jointly with E E ?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "UW CSE Lookup",
  "action_input": "What CSE courses are jointly offered with EE?"
}
``` 

Observation: According to the given information, only CSE 561: Computer Communications And Networks is jointly offered with EE.


> Finished chain.


'According to the given information, only CSE 561: Computer Communications And Networks is jointly offered with EE.'

# The Power of Agents

Finally, let's highlight exactly how powerful Agents can be by dynamically creating a Chain using various tools at each step of the process.

In [58]:
chatbot.respond("Who is the president of France married to? What is their age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Wikipedia",
  "action_input": "President of France spouse"
}
```


Observation: Page: List of spouses or partners of the president of France
Summary: Spouses and partners of the president of France often play a protocol role at the Élysée Palace and during official visits, though they possess no official title. Brigitte Macron is the spouse of the current president, Emmanuel Macron, who took office on 14 May 2017.

Page: President of the People's Republic of China
Summary: The president of the People's Republic of China, commonly called the president of China, is the head of state and the second-highest political office of the People's Republic of China. The presidency on its own is a ceremonial office and has no real power in China's political system. However, the post has been held by the general secretary of the Chinese Communist Party and chairman of the Central Military Commission since 1993, who is China's de fact

"Brigitte Macron's age raised to the 0.43 power is approximately 6.14."

This amazing! The Agent is able to break down a complex task into smaller pieces and use the appropriate tool at each step, chaining together the output/input for successive steps. 